## Load 'jupyter notebook' from Conda Terminal before beginning to use GPU

In [1]:

from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.optimizers import SGD
import numpy as np
import tensorflow as tf

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print('Tensorflow: ', tf.__version__)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9979241503233483015
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4952306483
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8505219895480972799
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
Tensorflow:  1.11.0


In [32]:
X_train = np.load("X_train.dat")
y_train = np.load("y_train.dat")

In [33]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=2)

In [34]:
X_train = X_train/255

print(X_train.shape)
print(y_train.shape)

# split data into validation set
X_train_validation = X_train[850:, :,:,:]
y_train_validation = y_train[850:, :]
y_train = y_train[:850,:]
X_train = X_train[:850, :, :, :]
print("Validation x train set:" + str(X_train_validation.shape))
print("X train set:" + str(X_train.shape))
print("Validation y train set:" + str(y_train_validation.shape))
print("Y train set:" + str(y_train.shape))

(900, 124, 174, 3)
(900, 10)
Validation x train set:(50, 124, 174, 3)
X train set:(850, 124, 174, 3)
Validation y train set:(50, 10)
Y train set:(850, 10)


## Build the CNN network

In [52]:
# VGG-16 like network from Andrew Ng's course on Coursera

model = Sequential()

# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(64, (3, 3), strides=(1,1), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), strides=(1,1), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), strides=(1,1)))
model.add(Activation('relu'))
          
# Max pooling && Dropout
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.8))

# # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
model.add(Flatten())

model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.2))

model.add(Dense(10, activation='softmax'))

## Begin training the model

In [56]:
model.compile('adam', 'categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=64, epochs=100 , validation_data=(X_train_validation, y_train_validation))

Train on 850 samples, validate on 50 samples
Epoch 1/100
850/850 [==============================] - 3s 4ms/step - loss: 0.4644 - acc: 0.8553 - val_loss: 1.7178 - val_acc: 0.5800
Epoch 2/100
850/850 [==============================] - 2s 2ms/step - loss: 0.3228 - acc: 0.9118 - val_loss: 1.9436 - val_acc: 0.5200
Epoch 3/100
850/850 [==============================] - 2s 2ms/step - loss: 0.1683 - acc: 0.9588 - val_loss: 2.2905 - val_acc: 0.4800
Epoch 4/100
850/850 [==============================] - 2s 2ms/step - loss: 0.1406 - acc: 0.9600 - val_loss: 2.4402 - val_acc: 0.4600
Epoch 5/100
850/850 [==============================] - 2s 2ms/step - loss: 0.1558 - acc: 0.9600 - val_loss: 2.5706 - val_acc: 0.5000
Epoch 6/100
850/850 [==============================] - 2s 2ms/step - loss: 0.1047 - acc: 0.9729 - val_loss: 2.3640 - val_acc: 0.5600
Epoch 7/100
850/850 [==============================] - 2s 2ms/step - loss: 0.0663 - acc: 0.9882 - val_loss: 2.6825 - val_acc: 0.5200
Epoch 8/100
850/850 [===

KeyboardInterrupt: 

In [58]:
model.save("trained_music_classifier.h5")

## Evaluate the model

In [57]:
score = model.evaluate(X_train_validation, y_train_validation, batch_size=32, verbose=1)
print(score)

50/50 [==============================] - 0s 1ms/step
[3.419465198516846, 0.5800000071525574]


In [51]:
from keras import backend as K
K.clear_session()